# Data Analysis of Scraped Tweets

The JSON data is first read and useful attributes are extracted. This information can be used to extract sentiments around the keyword.

In [1]:
import json
import pandas as pd
from twitter_trends import TwitterBot

In [2]:
bot = TwitterBot()
bot.fetch_data(['covid', 'corona'], max_tweets = 1000) #Scraped data is stored at data/tweets.json

Getting tweet #1
Getting tweet #2
Getting tweet #3
Getting tweet #4
Getting tweet #5
Getting tweet #6
Getting tweet #7
Getting tweet #8
Getting tweet #9
Getting tweet #10
Getting tweet #11
Getting tweet #12
Getting tweet #13
Getting tweet #14
Getting tweet #15
Getting tweet #16
Getting tweet #17
Getting tweet #18
Getting tweet #19
Getting tweet #20
Getting tweet #21
Getting tweet #22
Getting tweet #23
Getting tweet #24
Getting tweet #25
Getting tweet #26
Getting tweet #27
Getting tweet #28
Getting tweet #29
Getting tweet #30
Getting tweet #31
Getting tweet #32
Getting tweet #33
Getting tweet #34
Getting tweet #35
Getting tweet #36
Getting tweet #37
Getting tweet #38
Getting tweet #39
Getting tweet #40
Getting tweet #41
Getting tweet #42
Getting tweet #43
Getting tweet #44
Getting tweet #45
Getting tweet #46
Getting tweet #47
Getting tweet #48
Getting tweet #49
Getting tweet #50
Getting tweet #51
Getting tweet #52
Getting tweet #53
Getting tweet #54
Getting tweet #55
Getting tweet #56
G

Getting tweet #443
Getting tweet #444
Getting tweet #445
Getting tweet #446
Getting tweet #447
Getting tweet #448
Getting tweet #449
Getting tweet #450
Getting tweet #451
Getting tweet #452
Getting tweet #453
Getting tweet #454
Getting tweet #455
Getting tweet #456
Getting tweet #457
Getting tweet #458
Getting tweet #459
Getting tweet #460
Getting tweet #461
Getting tweet #462
Getting tweet #463
Getting tweet #464
Getting tweet #465
Getting tweet #466
Getting tweet #467
Getting tweet #468
Getting tweet #469
Getting tweet #470
Getting tweet #471
Getting tweet #472
Getting tweet #473
Getting tweet #474
Getting tweet #475
Getting tweet #476
Getting tweet #477
Getting tweet #478
Getting tweet #479
Getting tweet #480
Getting tweet #481
Getting tweet #482
Getting tweet #483
Getting tweet #484
Getting tweet #485
Getting tweet #486
Getting tweet #487
Getting tweet #488
Getting tweet #489
Getting tweet #490
Getting tweet #491
Getting tweet #492
Getting tweet #493
Getting tweet #494
Getting twee

Getting tweet #887
Getting tweet #888
Getting tweet #889
Getting tweet #890
Getting tweet #891
Getting tweet #892
Getting tweet #893
Getting tweet #894
Getting tweet #895
Getting tweet #896
Getting tweet #897
Getting tweet #898
Getting tweet #899
Getting tweet #900
Getting tweet #901
Getting tweet #902
Getting tweet #903
Getting tweet #904
Getting tweet #905
Getting tweet #906
Getting tweet #907
Getting tweet #908
Getting tweet #909
Getting tweet #910
Getting tweet #911
Getting tweet #912
Getting tweet #913
Getting tweet #914
Getting tweet #915
Getting tweet #916
Getting tweet #917
Getting tweet #918
Getting tweet #919
Getting tweet #920
Getting tweet #921
Getting tweet #922
Getting tweet #923
Getting tweet #924
Getting tweet #925
Getting tweet #926
Getting tweet #927
Getting tweet #928
Getting tweet #929
Getting tweet #930
Getting tweet #931
Getting tweet #932
Getting tweet #933
Getting tweet #934
Getting tweet #935
Getting tweet #936
Getting tweet #937
Getting tweet #938
Getting twee

In [3]:
with open('data/tweets.json') as f:
    d = f.read()
    tweets = json.loads(d) #tweets is a list of dictionary

The possible attributes in the tweets are:

['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
'in_reply_to_status_id', 'in_reply_to_status_id_str',
'in_reply_to_user_id', 'in_reply_to_user_id_str',
'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
'contributors', 'retweeted_status', 'quoted_status_id',
'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink',
'is_quote_status', 'quote_count', 'reply_count', 'retweet_count',
'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
'lang', 'timestamp_ms', 'possibly_sensitive', 'extended_entities',
'display_text_range', 'extended_tweet']

Note that not all tweets would have all of these attributes present. Some attributes such as extended_tweet may be present only in fraction of tweets.

In [4]:
processed = []
for tweet in tweets:
    if tweet['lang'] != 'en':
        continue
    t = {}
    t['created_at'] = tweet['created_at']
    t['text'] = tweet['text']
    if 'extended_tweet' in tweet.keys():
        t['text'] = tweet['extended_tweet']['full_text']
    
    processed.append(t)

In [5]:
df = pd.DataFrame(processed)

In [6]:
df.head()

,created_at,text
0,Sun Jun 07 13:17:10 +0000 2020,RT @jrockstrom: Want to reduce risks of anothe...
1,Sun Jun 07 13:17:10 +0000 2020,RT @josungkim: More coronavirus worries for So...
2,Sun Jun 07 13:17:11 +0000 2020,Covid lowkey fell off
3,Sun Jun 07 13:17:11 +0000 2020,RT @AmieAslam: It is interesting to analyze th...
4,Sun Jun 07 13:17:11 +0000 2020,RT @SkyNewsBreak: Scotland and Northern Irelan...


In [7]:
#Read the data in csv file
df = pd.read_csv('data/tweets.csv')
df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_entities,possibly_sensitive,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,Sun Jun 07 13:17:10 +0000 2020,1269619412396580864,1269619412396580864,RT @jrockstrom: Want to reduce risks of anothe...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,en,1591535830960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Jun 07 13:17:10 +0000 2020,1269619412459495424,1269619412459495424,RT @marioadolfo: O Inpe divulgou dados de desm...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,pt,1591535830975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sun Jun 07 13:17:10 +0000 2020,1269619412530630656,1269619412530630656,RT @josungkim: More coronavirus worries for So...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,NaN,...,en,1591535830992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sun Jun 07 13:17:11 +0000 2020,1269619412576993280,1269619412576993280,Covid lowkey fell off,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,en,1591535831003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sun Jun 07 13:17:11 +0000 2020,1269619412639920128,1269619412639920128,RT @AmieAslam: It is interesting to analyze th...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,1591535831018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Description
df.describe()

,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,geo,coordinates,contributors,quote_count,reply_count,retweet_count,favorite_count,timestamp_ms,quoted_status_id,quoted_status_id_str
count,1.000000e+03,1.000000e+03,8.900000e+01,8.900000e+01,9.400000e+01,9.400000e+01,0.0,0.0,0.0,1000.0,1000.0,1000.0,1000.0,1.000000e+03,2.320000e+02,2.320000e+02
mean,1.269619e+18,1.269619e+18,1.269521e+18,1.269521e+18,3.376868e+17,3.376868e+17,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.591536e+12,1.268734e+18,1.268734e+18
std,2.933526e+10,2.933526e+10,3.558494e+14,3.558494e+14,4.954554e+17,4.954554e+17,NaN,NaN,NaN,0.0,0.0,0.0,0.0,6.994072e+03,4.300387e+15,4.300387e+15
min,1.269619e+18,1.269619e+18,1.266365e+18,1.266365e+18,1.917731e+06,1.917731e+06,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.591536e+12,1.231521e+18,1.231521e+18
25%,1.269619e+18,1.269619e+18,1.269548e+18,1.269548e+18,1.054703e+08,1.054703e+08,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.591536e+12,1.269169e+18,1.269169e+18
50%,1.269619e+18,1.269619e+18,1.269611e+18,1.269611e+18,6.881113e+08,6.881113e+08,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.591536e+12,1.269377e+18,1.269377e+18
75%,1.269619e+18,1.269619e+18,1.269618e+18,1.269618e+18,8.391224e+17,8.391224e+17,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.591536e+12,1.269581e+18,1.269581e+18
max,1.269620e+18,1.269620e+18,1.269619e+18,1.269619e+18,1.260186e+18,1.260186e+18,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.591536e+12,1.269619e+18,1.269619e+18


In [9]:
#Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
created_at                   1000 non-null object
id                           1000 non-null int64
id_str                       1000 non-null int64
text                         1000 non-null object
source                       1000 non-null object
truncated                    1000 non-null bool
in_reply_to_status_id        89 non-null float64
in_reply_to_status_id_str    89 non-null float64
in_reply_to_user_id          94 non-null float64
in_reply_to_user_id_str      94 non-null float64
in_reply_to_screen_name      94 non-null object
user                         1000 non-null object
geo                          0 non-null float64
coordinates                  0 non-null float64
place                        6 non-null object
contributors                 0 non-null float64
retweeted_status             725 non-null object
is_quote_status              1000 non-null bool
quote_count     